In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

<ipython-input-2-63192117ceec>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/Mean_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
   # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    

    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)

    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

    # Xây dựng mô hình 4-layer stacked LSTM
    def build_model(hp):
        model = tf.keras.Sequential()
        
        # LSTM layer 1
        model.add(layers.LSTM(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True,
            input_shape=(X_train_res.shape[1], 1)
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 2
        model.add(layers.LSTM(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 3
        model.add(layers.LSTM(
            units=hp.Int('units_3', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 4
        model.add(layers.LSTM(
            units=hp.Int('units_4', min_value=32, max_value=256, step=32),
            return_sequences=False
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_4', min_value=0.1, max_value=0.5, step=0.1)))

        # Lớp đầu ra
        model.add(layers.Dense(5, activation='softmax'))

        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        return model

    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'lstm_tuning_week{week_number}'
    )

    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)

    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params


In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 04m 23s]
val_accuracy: 0.9987797141075134

Best val_accuracy So Far: 0.9993898868560791
Total elapsed time: 00h 45m 03s
Best Parameters for Week 1:
units_1: 96
dropout_1: 0.1
units_2: 128
dropout_2: 0.2
units_3: 128
dropout_3: 0.2
units_4: 32
dropout_4: 0.30000000000000004
learning_rate: 0.000700317602035526

Best Parameters for Week 2:
units_1: 256
dropout_1: 0.30000000000000004
units_2: 32
dropout_2: 0.5
units_3: 160
dropout_3: 0.30000000000000004
units_4: 96
dropout_4: 0.30000000000000004
learning_rate: 0.0009719978423652108

Best Parameters for Week 3:
units_1: 32
dropout_1: 0.30000000000000004
units_2: 96
dropout_2: 0.30000000000000004
units_3: 256
dropout_3: 0.1
units_4: 256
dropout_4: 0.4
learning_rate: 0.0010820242763584396

Best Parameters for Week 4:
units_1: 192
dropout_1: 0.30000000000000004
units_2: 224
dropout_2: 0.1
units_3: 224
dropout_3: 0.30000000000000004
units_4: 160
dropout_4: 0.2
learning_rate: 0.0012776325177753622


In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        model = build_lstm_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 96, 'dropout_1': 0.1, 'units_2': 128, 'dropout_2': 0.2, 'units_3': 128, 'dropout_3': 0.2, 'units_4': 32, 'dropout_4': 0.30000000000000004, 'learning_rate': 0.000700317602035526}
Fold 1: Using file /kaggle/input/Mean_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6010 - loss: 0.1681 - val_accuracy: 0.6748 - val_loss: 0.1187
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6817 - loss: 0.1167 - val_accuracy: 0.7125 - val_loss: 0.1062
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7082 - loss: 0.1073 - val_accuracy: 0.7377 - val_loss: 0.0979
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7290 - loss: 0.1031 - val_accuracy: 0.7472 - val_loss: 0.0965
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7464 - loss: 0.1006 - val_accuracy: 0.7739 - val_loss: 0.0858
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7645 - loss: 0.0933 - val_accuracy: 0.7861 - val_loss: 0.0845
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7764 - loss: 0.0892 - val_accuracy: 0.7873 - val_loss: 0.0854
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7742 - loss: 0.0905 - val_accuracy: 0.7876 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6001 - loss: 0.1717 - val_accuracy: 0.6733 - val_loss: 0.1178
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6818 - loss: 0.1192 - val_accuracy: 0.7106 - val_loss: 0.1037
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7055 - loss: 0.1091 - val_accuracy: 0.7556 - val_loss: 0.0961
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7317 - loss: 0.1041 - val_accuracy: 0.7674 - val_loss: 0.0915
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7444 - loss: 0.0973 - val_accuracy: 0.7636 - val_loss: 0.0925
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7538 - loss: 0.0961 - val_accuracy: 0.7945 - val_loss: 0.0844
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7603 - loss: 0.0941 - val_accuracy: 0.8029 - val_loss: 0.0808
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7664 - loss: 0.0922 - val_accuracy: 0.8086 - val

...

Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6142 - loss: 0.1467 - val_accuracy: 0.7026 - val_loss: 0.1024
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7499 - loss: 0.0970 - val_accuracy: 0.8155 - val_loss: 0.0725
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8334 - loss: 0.0701 - val_accuracy: 0.9116 - val_loss: 0.0397
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8850 - loss: 0.0472 - val_accuracy: 0.8750 - val_loss: 0.0636
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8926 - loss: 0.0435 - val_accuracy: 0.9356 - val_loss: 0.0237
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9100 - loss: 0.0390 - val_accuracy: 0.9226 - val_loss: 0.0258
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9141 - loss: 0.0338 - val_accuracy: 0.9356 - val_loss: 0.0240
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9201 - loss: 0.0316 - val_accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6036 - loss: 0.1487 - val_accuracy: 0.7677 - val_loss: 0.0946
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7712 - loss: 0.0943 - val_accuracy: 0.8608 - val_loss: 0.0505
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8572 - loss: 0.0613 - val_accuracy: 0.9195 - val_loss: 0.0326
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8849 - loss: 0.0490 - val_accuracy: 0.9249 - val_loss: 0.0269
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8939 - loss: 0.0458 - val_accuracy: 0.9222 - val_loss: 0.0297
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9133 - loss: 0.0346 - val_accuracy: 0.9214 - val_loss: 0.0389
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9103 - loss: 0.0367 - val_accuracy: 0.9390 - val_loss: 0.0235
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9144 - loss: 0.0340 - val_accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6097 - loss: 0.1488 - val_accuracy: 0.7002 - val_loss: 0.1099
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7338 - loss: 0.1046 - val_accuracy: 0.8783 - val_loss: 0.0510
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8434 - loss: 0.0648 - val_accuracy: 0.8970 - val_loss: 0.0459
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8812 - loss: 0.0502 - val_accuracy: 0.9344 - val_loss: 0.0283
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8908 - loss: 0.0439 - val_accuracy: 0.9367 - val_loss: 0.0231
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9062 - loss: 0.0373 - val_accuracy: 0.9355 - val_loss: 0.0249
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9089 - loss: 0.0363 - val_accuracy: 0.9367 - val_loss: 0.0229
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9156 - loss: 0.0330 - val_accuracy: 0.9428 - val_

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.9617162108421325
Average Train Time: 136.8927 seconds
Average Test Time: 0.7436 seconds
Average AUC Macro: 0.9937957745331202
Average AUC Weighted: 0.9981129190508206

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.794235        0.751667          0.771092     0.941538
1      1           0.601165        0.391144          0.470465     0.931138
2      2           0.784121        0.465972          0.582378     0.911803
3      3           0.783742        0.584973          0.668896     0.934829
4      4           0.878781        0.970440          0.922211     0.966477

Average Confusion Matrix:
       0     1     2     3       4
0  451.0  14.2   8.6   9.8   116.4
1   29.2  34.2   2.4   3.2    18.4
2   37.0   5.8  76.6   6.4    38.6
3   23.6   1.6   2.4  97.8    41.8
4   29.6   2.0   8.0   7.8  1556.2

=== Results for week2 ===
Average Accura

## Kiểm tra trên tập test

In [11]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model

In [12]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    model = build_lstm_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [13]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.4796 - loss: 0.1960 - val_accuracy: 0.6787 - val_loss: 0.2172
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6140 - loss: 0.1544 - val_accuracy: 0.6770 - val_loss: 0.2083
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6405 - loss: 0.1436 - val_accuracy: 0.7174 - val_loss: 0.1921
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6529 - loss: 0.1383 - val_accuracy: 0.6875 - val_loss: 0.1672
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6655 - loss: 0.1326 - val_accuracy: 0.6835 - val_loss: 0.1694
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6775 - loss: 0.1272 - val_accuracy: 0.7221 - val_loss: 0.1475
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6832 - loss: 0.1238 - val_accuracy: 0.7216 - val_loss: 0.1453
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6892 - loss: 0.1197 - val_accura

In [14]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.4263 - loss: 0.2062 - val_accuracy: 0.7538 - val_loss: 0.1893
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5511 - loss: 0.1665 - val_accuracy: 0.7448 - val_loss: 0.1908
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5774 - loss: 0.1576 - val_accuracy: 0.6964 - val_loss: 0.1964
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5992 - loss: 0.1471 - val_accuracy: 0.7146 - val_loss: 0.1901
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6263 - loss: 0.1368 - val_accuracy: 0.6575 - val_loss: 0.2067
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6449 - loss: 0.1291 - val_accuracy: 0.6406 - val_loss: 0.2048
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6694 - loss: 0.1208 - val_accuracy: 0.6867 - val_loss: 0.1759
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6897 - loss: 0.1144 - val_accura

In [15]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.4054 - loss: 0.2087 - val_accuracy: 0.6964 - val_loss: 0.2187
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5996 - loss: 0.1485 - val_accuracy: 0.7216 - val_loss: 0.1636
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6949 - loss: 0.1109 - val_accuracy: 0.7291 - val_loss: 0.1361
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7373 - loss: 0.0929 - val_accuracy: 0.6648 - val_loss: 0.1553
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7596 - loss: 0.0853 - val_accuracy: 0.6605 - val_loss: 0.1574
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7732 - loss: 0.0786 - val_accuracy: 0.7114 - val_loss: 0.1244
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7846 - loss: 0.0732 - val_accuracy: 0.7802 - val_loss: 0.1054
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8005 - loss: 0.0677 - val_accura

In [16]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.5109 - loss: 0.1827 - val_accuracy: 0.7221 - val_loss: 0.1577
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7587 - loss: 0.0813 - val_accuracy: 0.5657 - val_loss: 0.1330
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8082 - loss: 0.0600 - val_accuracy: 0.7428 - val_loss: 0.1000
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8381 - loss: 0.0483 - val_accuracy: 0.7997 - val_loss: 0.0675
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8688 - loss: 0.0393 - val_accuracy: 0.9042 - val_loss: 0.0226
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9001 - loss: 0.0289 - val_accuracy: 0.8334 - val_loss: 0.0457
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9065 - loss: 0.0266 - val_accuracy: 0.9327 - val_loss: 0.0148
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9224 - loss: 0.0211 - val_accura

In [17]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 448.61 seconds
  Test Time: 0.68 seconds
  Accurancy: 0.7925564368517388
  Precision: [0.79790941 0.46753247 0.58       0.67058824 0.84311927]
  Recall: [0.61066667 0.65454545 0.5631068  0.54807692 0.91716567]
  F1-Score: [0.6918429  0.54545455 0.57142857 0.6031746  0.87858509]
  Macro Precision: 0.6718298753094272
  Macro Recall: 0.6587123018136448
  Macro F1-Score: 0.65809714128038
  Confusion Matrix:
[[229  22  18   2 104]
 [  3  36   0   2  14]
 [ 19   5  58   2  19]
 [  2   4   7  57  34]
 [ 34  10  17  22 919]]

=== AUC-ROC ===
AUC Macro: 0.8801
AUC Weighted: 0.8884
AUC per Label: [0.90398945 0.91183425 0.82705679 0.86813455 0.88970567]
Week 2:
  Train Time: 448.32 seconds
  Test Time: 0.68 seconds
  Accurancy: 0.7699816961561928
  Precision: [0.78995434 0.76923077 0.63461538 0.58139535 0.79227468]
  Recall: [0.46133333 0.90909091 0.6407767  0.48076923 0.92115768]
  F1-Score: [0.58249158 0.83333333 0.63768116 0.52631579